In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import scipy.stats as stats
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output, HTML
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind

In [2]:
data_EMGcine=pd.read_excel('Resultats_PowerBI_tot.xlsx', sheet_name=2)
data_timings=pd.read_excel('Resultats_PowerBI_tot.xlsx', sheet_name=3)

In [3]:
data_EMGcine.columns = data_EMGcine.columns.str.strip()
data_timings.columns = data_timings.columns.str.strip()

In [4]:
data=data_EMGcine

In [5]:
# Tabs
tab = widgets.Tab()
tab1_output = widgets.Output()
tab.children = [tab1_output]  # Ajouter la sortie avec les graphes et widgets dans le premier onglet
tab.set_title(0, "Tab1")  # Nom de l'onglet

with tab1_output:
    batteur_dropdown = widgets.Dropdown(
        options=data['Sujet'].unique(),
        description='Drummer:',
        style={'description_width': 'initial'}
    )

    vitesse_slider = widgets.IntSlider(
    min=data['Vitesse_bpm'].min(),
    max=data['Vitesse_bpm'].max(),
    step=10,
    description='Speed (bpm):',
    continuous_update=False,  # 🔹 Met à jour SEULEMENT quand on relâche la souris
    style={'description_width': 'initial', 'font_size': '16px'},  # Augmente la taille de la police du slider
    layout=widgets.Layout(width='600px', height='50px')  # Large et plus visible
    )

    vitesse_label = widgets.Label(
        "Select the trial speed :",
        style={'font_size': '18px'}  # Augmente la taille de la police du label
    )

    slider_box = widgets.VBox([
        vitesse_label,
        vitesse_slider
    ], layout=widgets.Layout(
        border='solid 2px black',  # Bordure visible
        padding='10px',            # Espacement intérieur
        margin='10px',             # Espacement extérieur
        width='650px'              # Ajuste la taille du cadre
    ))


    # Liste des colonnes dans l'ordre voulu
    ordered_columns = ["ERE_90th", "ABD_90th", "TFL_90th", "GluMed_90th", "RF_90th",
                       "Mean_trunk_flexion", "Mean_pelvis_anteversion", "MeanFlexion_Hip", 
                       "MeanAbduction_Hip", "MeanFlexion_Knee", "MeanFlexion_Ankle",
                       "AmpFlexionExtension_Hip", "AmpFlexionExtension_Ankle", "RATIO_ANKLE_HIP",
                       "AmpInternExternRot_Hip", "AmpInternExternRot_Ankle"]

    # Assure-toi que seules ces colonnes sont incluses et dans le bon ordre
    param_dropdown = widgets.Dropdown(
        options=[col for col in ordered_columns if col in data.columns],  # Sélectionne les colonnes valides
        description="Parameter:"
    )

    display(batteur_dropdown,slider_box,param_dropdown)


In [6]:
def update_graphs(batteur, vitesse, parametre):
    """
    Met à jour les graphiques pour afficher :
    1. Comparaison entre le batteur sélectionné et la moyenne du groupe, filtrée par la vitesse.
    2. Évolution du paramètre en fonction de la vitesse, avec toutes les vitesses.
    """
   

     # 🟢 Filtrer les données pour le batteur sélectionné et la vitesse
    df_batteur_gauche = data[(data["Sujet"] == batteur) & (data["Vitesse_bpm"] == vitesse) & (data["Cote"] == 1)]
    df_batteur_droite = data[(data["Sujet"] == batteur) & (data["Vitesse_bpm"] == vitesse) & (data["Cote"] == 2)]


    df_batteur_tot_gauche = data[(data["Sujet"] == batteur) & (data["Cote"] == 1)]
    df_batteur_tot_droite = data[(data["Sujet"] == batteur) & (data["Cote"] == 2)]
    
    #df_batteur_tot = df_batteur_tot.dropna()  # Supprime les lignes contenant des NaN

    # 🟢 Filtrer les données du groupe par la vitesse et les côtés sélectionnés
    filtered_df = data[(data["Vitesse_bpm"] == vitesse)]
    #filtered_df = filtered_df.dropna()  # Supprime les lignes contenant des NaN

    filtered_df_tot = data
    #filtered_df_tot = filtered_df_tot.dropna()  # Supprime les lignes contenant des NaN

    # Calcul de la moyenne et de l'écart-type du groupe
    group_mean = filtered_df.drop(columns=["Sujet", "Vitesse_bpm", "Cote"]).mean()
    group_std = filtered_df.drop(columns=["Sujet", "Vitesse_bpm", "Cote"]).std()


    vitesse_reelle_batteur = df_batteur_gauche["REALBPM"].mean()
    vitesse_relle_batteur_text = f"**Real speed** : {vitesse_reelle_batteur: .0f} BPM"
    
    precision_batteur = 100 - abs((vitesse-df_batteur_gauche["REALBPM"].mean())/vitesse)*100
    precision_batteur_text = f"**Speed precision** : {precision_batteur: .0f}\\%"

    
    cv_batteur_gauche = abs(df_batteur_gauche[parametre].std() / df_batteur_gauche[parametre].mean()) * 100
    cv_batteur_droite = abs(df_batteur_droite[parametre].std() / df_batteur_droite[parametre].mean()) * 100
    
    cv_batteur_gauche_text = f"**Regularity of the parameter for left side** : {100-cv_batteur_gauche: .0f}\\%"
    cv_batteur_droite_text = f"**Regularity of the parameter for right side** : {100-cv_batteur_droite: .0f}\\%"

    # Texte paramètre 
    if parametre == "ERE_90th":
        text_explication = """
        <strong>90th percentile of activation of Erector Spinae muscle (%)</strong> : <br>
        Erector Spinae is mainly responsible for trunk extension, that when the back is straightened, moving the upper body backward. <br>
        This value represents the highest activation of the Erector Spinae muscle, measured during trial cycles, displayed in percentage of the maximal activation of the muscle (measured during Maximal Voluntary Contractions). <br> 
        This means that if the value is 50% during drumming, the drummer is activating his muscle at 50% of the maximal contraction he can do using this muscle. <br> 
        The higher this value during drumming, the higher the risk for musculoskeletal disorders developing in the lower back, for example leading to sciatic nerve injury. <br>
        Two ways to reduce this value : <br>
        1. Playing in a more relaxed manner: directly reducing the activation level during drumming.<br>
        2. Strengthening this muscle with exercises: by increasing its maximal value, the level of activation during drumming is therefore reduced.
        """
        image_path = "images/erectorspinae.png"
    elif parametre == "GluMed_90th":
        text_explication = """
        <strong>90th percentile of activation of Gluteus Medius muscle (%)</strong> : <br>
        Gluteus Medius is mainly responsible for hip abduction, that is the movement of the leg away from the center of the body. <br>
        This value represents the highest activation of the Gluteus Medius muscle, measured during trial cycles, displayed in percentage of the maximal activation of the muscle (measured during Maximal Voluntary Contractions). <br> 
        This means that if the value is 50% during drumming, the drummer is activating his muscle at 50% of the maximal contraction he can do using this muscle. <br> 
        The higher this value during drumming, the higher the risk for musculoskeletal disorders developing in the buttocks, for example leading to piriformis syndrome, which in turn can lead to sciatic nerve injury. <br>
        Two ways to reduce this value : <br>
        1. Playing in a more relaxed manner: directly reducing the activation level during drumming.<br>
        2. Strengthening this muscle with exercises: by increasing its maximal value, the level of activation during drumming is therefore reduced.
        """        
        image_path = "images/gluteusmedius.png"
    elif parametre == "RF_90th":
        text_explication = """
        <strong>90th percentile of activation of Rectus Femoris muscle (%)</strong> : <br>
        Rectus Femoris is mainly responsible for hip flexion, that is when the leg is lifted up, bending at the hip; and for knee extension, that is when the leg is straightened, moving the lower leg away from the thigh.<br>
        This value represents the highest activation of the Rectus Femoris muscle, measured during trial cycles, displayed in percentage of the maximal activation of the muscle (measured during Maximal Voluntary Contractions). <br> 
        This means that if the value is 50% during drumming, the drummer is activating his muscle at 50% of the maximal contraction he can do using this muscle. <br> 
        The higher this value during drumming, the higher the risk for musculoskeletal disorders developing in the hip and knee area, for example leading to tendon strain. <br>
        Two ways to reduce this value : <br>
        1. Playing in a more relaxed manner: directly reducing the activation level during drumming.<br>
        2. Strengthening this muscle with exercises: by increasing its maximal value, the level of activation during drumming is therefore reduced.
        """        
        image_path = "images/rectusfemoris.png"
    elif parametre == "TFL_90th":
        text_explication = """
        <strong>90th percentile of activation of Tensor Fascia Latae muscle (%)</strong> : <br>
        Tensor Fascia Latae is mainly responsible for hip flexion, that is when the leg is lifted up, bending at the hip; and hip abduction, that is the movement of the leg away from the center of the body.<br>
        This value represents the highest activation of the Tensor Fascia Latae muscle, measured during trial cycles, displayed in percentage of the maximal activation of the muscle (measured during Maximal Voluntary Contractions). <br> 
        This means that if the value is 50% during drumming, the drummer is activating his muscle at 50% of the maximal contraction he can do using this muscle. <br> 
        The higher this value during drumming, the higher the risk for musculoskeletal disorders developing in the hips area, for example leading to tendinitis and iliotibial band syndrome. <br>
        Two ways to reduce this value : <br>
        1. Playing in a more relaxed manner: directly reducing the activation level during drumming.<br>
        2. Strengthening this muscle with exercises: by increasing its maximal value, the level of activation during drumming is therefore reduced.
        """        
        image_path = "images/tensorfascialatae.png"
    elif parametre == "ABD_90th":
        text_explication = """
        <strong>90th percentile of activation of Rectus Abdominis muscle (%)</strong> : <br>
        Rectus Abdominis is mainly responsible for trunk flexion, that is when the upper body is bent forward.<br>
        This value represents the highest activation of the Rectus Abdominis muscle, measured during trial cycles, displayed in percentage of the maximal activation of the muscle (measured during Maximal Voluntary Contractions). <br> 
        This means that if the value is 50% during drumming, the drummer is activating his muscle at 50% of the maximal contraction he can do using this muscle. <br> 
        This muscle does not present activation levels at risk for musculoskeletal disorders development during the ankle technique. <br>
        """        
        image_path = "images/rectusabdominis.png"
    
    elif parametre == "Mean_trunk_flexion":
        text_explication = """
        <strong>Mean position of Trunk Flexion (in degrees)</strong> : <br>
        Trunk flexion is when the upper body is bent forward.<br>
        This value is positive when the upper body is bent forward, and negative when the upper body is bent backwards (= trunk extension). <br>
        Position in excessive trunk flexion can lead to excessive activation of the Erector Spinae muscle. <br>
        """        
        image_path = "images/meantrunkflexion.png"
    elif parametre == "Mean_pelvis_anteversion":
        text_explication = """
        <strong>Mean position of Pelvis Flexion (or anteversion) (in degrees)</strong> : <br>
        Pelvis flexion or anteversion is the forward tilting of the pelvis. <br>
        This value is positive when the pelvis is tilting forward, and negative when the pelvis is tilting backwards (= pelvis retroversion). <br>
        Position in excessive pelvis anteversion can lead to lumbar hyperlordosis and is not optimal for hip flexor muscles' tension and force. However excessive pelvis retroversion can also lead to lower back pain. <br>
        """        
        image_path = "images/meanpelvisflexion.png"
    elif parametre == "MeanFlexion_Hip":
        text_explication = """
        <strong>Mean position of Hip Flexion (in degrees)</strong> : <br>
        Hip flexion is when the leg is lifted up, bending at the hip. <br>
        This value is 0 when the thigh is perpendicular to the pelvis, and positive when the hip is flexed forward. <br>
        Drummers using the ankle technique usually seat with the hips higher than the knees, meaning in a flexed position of the hip <90°. Lower position with hip flexion close to 90° allows for more power at the hip level for lower drumming speeds.<br>
        """        
        image_path = "images/meanhipflexion.png"
    elif parametre == "MeanFlexion_Knee":
        text_explication = """
        <strong>Mean position of Knee Flexion (in degrees)</strong> : <br>
        Knee flexion is when the leg is flexed, moving the lower leg closer to the thigh. <br>
        This value is 0 when the thigh is aligned with the lower leg, and positive when the lower leg is moved closer to the thigh. <br>
        The knee flexion value is closely related to the hip flexion.<br>
        """        
        image_path = "images/meankneeflexion.png"
    elif parametre == "MeanFlexion_Ankle":
        text_explication = """
        <strong>Mean position of Ankle Flexion (or dorsiflexion) (in degrees)</strong> : <br>
        Ankle flexion is when the ankle is flexed, moving the toes closer to the lower leg. <br>
        This value is 0 when the ankle is perpendicular to the lower leg, positive when the toes are closer to the lower leg (dorsiflexion), and negative when the foot is moved downwards, pointing the toes away from the body (=plantar flexion). <br>
        Usually, we observe that with increasing speed, the ankle flexion is reduced, and goes from positive values to negative values. In drummers using only the trembling of the ankle with reduced control, the foot is generally moved downwards (higher negative values) as the tibialis anterior muscle is not being used for the ankle technique. In expert drummers, we observe a more neutral position of the ankle (close to 0). <br>
        """        
        image_path = "images/meanankleflexion.png"
    elif parametre == "MeanAbduction_Hip":
        text_explication = """
        <strong>Mean position of Hip Abduction (in degrees)</strong> : <br>
        Hip Abduction is the movement of the leg away from the center of the body. <br>
        This value is positive when the leg is moved away from the center of the body, and negative when the leg is brought back toward the middle of the body. <br>
        This value is very dependent on the size of the snare and the length of the drive shaft of the pedal. <br>
        """        
        image_path = "images/meanhipabduction.png"
    elif parametre == "AmpFlexionExtension_Hip":
        text_explication = """
        <strong>Amplitude of Hip Flexion Extension (in degrees)</strong> : <br>
        The amplitude of hip flexion extension refers to the amount of movement between the two extreme positions of the hip.<br>
        The amplitude measures the total range of motion between these two positions, from the maximum flexion (leg up) to the maximum extension (leg down). <br>
        The larger the movement between these extremes, the larger the amplitude.<br>
        During the ankle technique, hip flexion extension amplitude is reduced, in comparison to full leg motion.<br>
        Some mixed ankle / hip techniques can use both ankle and hip flexion extension movements. <br>
        """        
        image_path = "images/hipflexionextension.webm"
    elif parametre == "AmpFlexionExtension_Ankle":
        text_explication = """
        <strong>Amplitude of Ankle Flexion Extension (in degrees)</strong> : <br>
        The amplitude of ankle flexion extension refers to the amount of movement between the two extreme positions of the ankle.<br>
        The amplitude measures the total range of motion between these two positions, from the maximum flexion (foot up) to the maximum extension (foot down). <br>
        The larger the movement between these extremes, the larger the amplitude.<br>
        During the ankle technique, a larger amplitude of ankle flexion extension concurs with a better control of the technique.<br>
        Usually, the amplitude is reduced with increasing trial speed.<br>
        """        
        image_path = "images/ankleflexionextension.webm"
    elif parametre == "RATIO_ANKLE_HIP":
        text_explication = """
        <strong>Ratio of Ankle Flexion Extension Amplitude over Hip Flexion Extension Amplitude (in degrees)</strong> : <br>
        This ratio represents the division of ankle flexion extension amplitude by the hip flexion extension amplitude.<br>
        The higher this ratio, the higher the role of the ankle in comparison to the hip in the drumming technique. <br>
        Drummers using only the ankle technique will have higher ratios than drummers using a mixed hip / ankle technique, who will have lower ratios.<br>
        """        
        image_path = "images/blank.png"
    elif parametre == "AmpInternExternRot_Hip":
        text_explication = """
        <strong>Amplitude of Hip Internal External Rotation (in degrees)</strong> : <br>
        The amplitude of hip internal external rotation refers to the amount of movement between the two extreme positions of rotation of the hip.<br>
        The amplitude measures the total range of motion between these two positions, from the maximum external rotation to the maximum internal rotation. <br>
        The larger the movement between these extremes, the larger the amplitude.<br>
        The swivel technique is associated to higher hip internal external rotation amplitude, alongside with higher internal external rotation amplitude of the ankle, in comparison to more vertical techniques. <br>
        """        
        image_path = "images/hiprotation.webm"
    elif parametre == "AmpInternExternRot_Ankle":
        text_explication = """
        <strong>Amplitude of Ankle Internal External Rotation (in degrees)</strong> : <br>
        The amplitude of ankle internal external rotation refers to the amount of movement between the two extreme positions of rotation of the ankle.<br>
        The amplitude measures the total range of motion between these two positions, from the maximum external rotation to the maximum internal rotation. <br>
        The larger the movement between these extremes, the larger the amplitude.<br>
        The swivel technique is associated to higher ankle internal external rotation amplitude, alongside with higher internal external rotation amplitude of the hip, in comparison to more vertical techniques. <br>
        """        
        image_path = "images/anklerotation.webm"
    else:
        text_explication="<strong>Pas encore fait</strong>"
        image_path = "images/default.png"

    if parametre == "AmpFlexionExtension_Hip" or parametre == "AmpFlexionExtension_Ankle" or parametre == "AmpInternExternRot_Hip" or parametre == "AmpInternExternRot_Ankle" : 
        # Générer le HTML avec une mise en page en ligne (flexbox)
        html_code = f"""
        <div style="display: flex; align-items: center;">
            <video src="{image_path}" width="300px" style="margin-right: 20px;" loop autoplay>
                Your browser does not support the video tag.
            </video>
            <p style="font-size: 16px;">{text_explication}</p>
        </div>
        """

    else:
        # Générer le HTML avec une mise en page en ligne (flexbox)
        html_code = f"""
        <div style="display: flex; align-items: center;">
            <img src="{image_path}" width="300px" style="margin-right: 20px;">
            <p style="font-size: 16px;">{text_explication}</p>
        </div>
        """

    #explications données régularité et symétrie
    regularite_symmetry_explications = "Regularity percentage represents how consistent or regular the measured parameter is. It shows how much the values for each bass drum hits vary compared to the average. \n\n Symmetry percentage represents how similar the values are for the left and right sides."
    
    # Vérification si des données existent pour chaque côté avant de calculer la moyenne
    if df_batteur_gauche.empty or df_batteur_droite.empty:
        symmetrie_batteur_text = "**Parameter symmetry index** : Unable to calculate (missing data)."
    else:
        # Calcul des moyennes en ignorant les NaN
        mean_gauche = df_batteur_gauche[parametre].mean()
        mean_droite = df_batteur_droite[parametre].mean()
    
        asymmetrie_batteur = abs(mean_gauche - mean_droite) / max(mean_droite,mean_gauche) * 100
        symmetrie_batteur_text = f"**Parameter symmetry index** : {100-asymmetrie_batteur:.0f}\\%"

    
    # GAUCHE TEST STATISTIQUE
    if not df_batteur_gauche.empty and not group_mean.empty:
        # Test t de Student
        t_stat_gauche, p_value_gauche = ttest_ind(df_batteur_gauche[parametre], filtered_df[parametre], nan_policy='omit', equal_var=False)

        # Vérifier si le p-value est valide
        if np.isnan(p_value_gauche):
            p_value_gauche = "Missing data for stat test"
        else:
            p_value_gauche = f"p = {p_value_gauche:.5f}"  # Afficher avec 5 décimales
    else:
        p_value_gauche = "Missing data for stat test"

     # DROITE TEST STATISTIQUE
    if not df_batteur_droite.empty and not group_mean.empty:
        # Test t de Student
        t_stat_droite, p_value_droite = ttest_ind(df_batteur_droite[parametre], filtered_df[parametre], nan_policy='omit', equal_var=False)

        # Vérifier si le p-value est valide
        if np.isnan(p_value_droite):
            p_value_droite = "Missing data for stat test"
        else:
            p_value_droite = f"p = {p_value_droite:.5f}"  # Afficher avec 5 décimales
    else:
        p_value_droite = "Missing data for stat test"

    # FIGURES EXPLICATIONS 

    figure1_explications = "In the following graph are displayed the values of the parameter for all bass drum hits performed during each of the ankle technique trials of the drummer (blue dots for right side, red dots for left side). The average curves are displayed for better visualization, along with the group average (black) for comparison purposes."
    figure2_explications = "Following the choice of the ankle technique trial speed, the following graph displays the average value of the parameter for the drummer on the left side (red), on the right side (blue), and for the group (black). A statistical test is performed to compare the values of the drummer with the group average, for both left and right sides. Please note that is the p value is <0.05, there is a statistical difference between the drummer and the group average, meaning that the drummer presents higher or lower values that the group average. If the p value is >0.05, there is no statistical difference, meaning that the group is within the average values. "

    
    # 🔹 FIGURE 1 : Comparaison du batteur avec la moyenne du groupe (Bar chart)
    #fig, (ax2, ax1) = plt.subplots(figsize=(15, 5), ncols=2)
    fig1 = plt.figure(figsize=(7, 5))  # Crée une nouvelle figure
    ax1 = fig1.add_subplot(111)  # Ajoute un seul axe à cette figure
    fig2 = plt.figure(figsize=(7, 5))  # Crée une nouvelle figure
    ax2 = fig2.add_subplot(111)  # Ajoute un seul axe à cette figure

    ##FIGURE 1
    # 📌 Tracé des données brutes pour le batteur sur toutes les vitesses
    sns.scatterplot(x=df_batteur_tot_gauche["Vitesse_bpm"], y=df_batteur_tot_gauche[parametre], color="red", ax=ax1)
    sns.scatterplot(x=df_batteur_tot_droite["Vitesse_bpm"], y=df_batteur_tot_droite[parametre], color="blue", ax=ax1)
    
    # 📌 Tracé de la courbe moyenne pour le batteur (sur toutes les vitesses)
    mean_batteur_by_speed_gauche = df_batteur_tot_gauche.groupby("Vitesse_bpm")[parametre].mean()
    sns.lineplot(x=mean_batteur_by_speed_gauche.index, y=mean_batteur_by_speed_gauche.values, color="red", linestyle="--", ax=ax1, label=f"Average for drummer {batteur} left side")

    mean_batteur_by_speed_droite = df_batteur_tot_droite.groupby("Vitesse_bpm")[parametre].mean()
    sns.lineplot(x=mean_batteur_by_speed_droite.index, y=mean_batteur_by_speed_droite.values, color="blue", linestyle="--", ax=ax1, label=f"Average for drummer {batteur} right side")
    
    # 📌 Tracé de la courbe moyenne du groupe (en prenant en compte toutes les vitesses)
    mean_group_by_speed = filtered_df_tot.groupby("Vitesse_bpm")[parametre].mean()
    sns.lineplot(x=mean_group_by_speed.index, y=mean_group_by_speed.values, color="black", linestyle="--", ax=ax1, label="Group average")
 
    # 📌 Mise en forme
    ax1.set_title(f"Evolution of {parametre} as function of speed")
    ax1.set_xlabel("Speed (bpm)")
    ax1.set_ylabel(parametre)
    ax1.legend()

    ## FIGURE 2
    # 📌 Tracé des barres pour le batteur et le groupe
    ax2.bar("Drummer left side", df_batteur_gauche[parametre].mean(), yerr=df_batteur_gauche[parametre].std(), color="red", alpha=0.6)
    ax2.bar("Group", group_mean[parametre], yerr=group_std[parametre], color="black", alpha=0.6)
    ax2.bar("Drummer right side", df_batteur_droite[parametre].mean(), yerr=df_batteur_droite[parametre].std(), color="blue", alpha=0.6)

    # 📌 Ajout du texte de la p-value
    ax2.text(0.05, 0.90, f"{p_value_gauche}", transform=ax1.transAxes, fontsize=12, verticalalignment='top', bbox=dict(facecolor='white', alpha=0.5))
    ax2.text(0.70, 0.90, f"{p_value_droite}", transform=ax1.transAxes, fontsize=12, verticalalignment='top', bbox=dict(facecolor='white', alpha=0.5))

    # 📌 Mise en forme
    ax2.set_title(f"Performance comparison on {parametre} for the speed {vitesse} bpm")
    ax2.set_ylabel(parametre)
    ax2.legend()


    with tab1_output:
        clear_output(wait=True)
        display(batteur_dropdown, param_dropdown)
        display(HTML(html_code))
        display(Markdown(f"### Following figure explanations \n{figure1_explications}"))
        display(fig1)
        display(slider_box)
        display(Markdown(f"### Speed precision \n{vitesse_relle_batteur_text} \n\n{precision_batteur_text}"))
        display(Markdown(f"### Following figure explanations \n{figure2_explications}"))
        display(fig2)
        display(Markdown(f"### Regularity and symmetry of the parameter \n{regularite_symmetry_explications} \n\n{cv_batteur_gauche_text} \n\n{cv_batteur_droite_text} \n\n{symmetrie_batteur_text}"))

        

In [7]:
widgets.interactive(update_graphs, 
                    batteur=batteur_dropdown, 
                    vitesse=vitesse_slider, 
                    parametre=param_dropdown)
tab